# Custom Losses and Metrics in Keras

Source of this notebook: https://www.tensorflow.org/tutorials/keras/regression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

##### Download the Auto-MPG Dataset

The goal of this task is to predict the fuel efficiency (Mileage per Gallon, MPG) of late-1970s and early 1980s cars. The input data for each car includes attributes like: cylinders, displacement, horsepower, weight, etc.

This dataset is not included in the `keras.datasets` so we will import it from a CSV file using `pandas` instead.

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

Let's make a copy (to keep the original untouched) and see the first rows

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()


Some basic preprocessing: eliminate rows with missing values, and convert the numeric `Origin` column to the corresponding geographic area. Finally, convert this categorical column to a one-hot representation using the `get_dummies` method.

In [ ]:
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

Create train and test subsets:

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


Split features from labels (the `pop()` method removes a column and creates a copy at the same time):

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

train_features.tail()

##### Build a simple model

We will approach the task with a simple linear model, built with the Sequential API. Before the single `Dense` layer, however, we will add a `Normalization()` layer, which normalizes each feature column to have 0 mean and unitary standard deviation.

We declare the normalization layer before the `Sequential()` call because we need to `adapt()` it to the input features statistics.

In [ ]:
normalizer = preprocessing.Normalization()

# convert the pandas df to a tensor-like numpy array
normalizer.adapt(np.array(train_features))

Then, we build a sequential model with a single linear layer. We can also use a multi-layer NN if we want.

In [ ]:
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

model.summary()
keras.utils.plot_model(model, 'mpg_regression.png', show_shapes=True)


##### Define a custom loss function

Let us define a custom loss function that we want to use to train this model. Here, we just re-define the Mean Squared Error (MSE), which of course is already present in `keras.losses`. However, in general, this is useful when you need to use task-specific or complex losses that are not already available.

Clearly, since the loss function has to be differentiated, we need to make sure that our custom definition only uses TensorFlow ops (or overloaded operators).

In [ ]:
def my_mse(y, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y))

Then, we can simply pass our custom `my_mse` loss to the compile function.

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss=my_mse, metrics=['mae'])

Then, we can train the model, normally.

In [ ]:
history = model.fit(
    train_features, train_labels,
    epochs=40,
    validation_split = 0.2,
    validation_freq=1
)

##### Use the history data structure to plot learning curves

Here we also show how to use the `history` data structure returned by `fit()` to plot the learning curves of our model .

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

##### Evaluate on test data

As usual, after training we evaluate the model on unseen data.

In [ ]:
test_results = model.evaluate(test_features, test_labels, verbose=1)


##### See the effect of normalization

Remember that we can call individual keras layers as functions. Let us try to see the effect of the `Normalization()` layer on one row of features.

In [ ]:
# don't use scientific notation
np.set_printoptions(suppress=True)

# note that we use 0:1 instead of 0 to make this a (1,9) tensor, and not a (9,) tensor
# the latter would not be processed correctly  by the normalizer
row = np.array(train_features)[0:1,:]
print(row)

norm_row = normalizer(row)
print(norm_row.numpy())

##### See the trained weights

Another aspect that we still didn't explore is how to inspect the internal weights of trained Keras layers. In this example, we have a single `Dense()` layer, which contains a $(9, 1)$ weight matrix, since it takes 9 columns as inputs and produces a single estimate as output. Let us see it.

The `.layers` field of our model contains a list of layers.

In [ ]:
print(model.layers)

Each (non custom) layer has a `weights` field, which contains a list of (typically named) tensors:

In [ ]:
# print(model.layers[1].weights)
print(model.layers[0].weights[0].numpy())